In [1]:
import sys
import binascii
import math
from functools import reduce
from os import urandom
from sha3 import keccak_256

from py_ecc.typing import Point2D, Field

Point = Point2D[Field]

quote = lambda x: '"' + str(x) + '"'
quotemany = lambda *x: ','.join(map(quote, x))
quotelist = lambda x: '[' + quotemany(*x) + ']'

safe_ord = ord if sys.version_info.major == 2 else lambda x: x if isinstance(x, int) else ord(x)

bytes_to_int = lambda x: reduce(lambda o, b: (o << 8) + safe_ord(b), [0] + list(x))

def packl(lnum):
    if lnum == 0:
        return b'\0'
    s = hex(lnum)[2:].rstrip('L')
    if len(s) & 1:
        s = '0' + s
    return binascii.unhexlify(s)

int_to_big_endian = packl

zpad = lambda x, l: b'\x00' * max(0, l - len(x)) + x

tobe256 = lambda v: zpad(int_to_big_endian(v), 32)

def hashs(*x):
    data = b''.join(map(tobe256, x))
    return bytes_to_int(keccak_256(data).digest())

randb256 = lambda: urandom(32)


bit_clear = lambda n, b: n ^ (1<<(b-1)) if n & 1<<(b-1) else n

bit_set = lambda n, b: n | (1<<(b-1))

bit_test = lambda n, b: 0 != (n & (1<<(b-1)))


def powmod(a,b,n):
    c = 0
    f = 1
    k = int(math.log(b, 2))
    while k >= 0:
        c *= 2
        f = (f*f)%n
        if b & (1 << k):
            c += 1
            f = (f*a) % n
        k -= 1
    return f


if __name__ == "__main__":
    assert bin(bit_clear(3, 1)) == '0b10'
    assert bin(bit_clear(3, 2)) == '0b1'
    assert bin(bit_set(0, 1)) == '0b1'


In [2]:
from random import randint

from py_ecc import bn128
from py_ecc.bn128 import add, multiply, curve_order, G1
from py_ecc.bn128 import field_modulus, FQ

# from .utils import hashs, bytes_to_int, powmod

asint = lambda x: x.n if isinstance(x, FQ) else x

randsn = lambda: randint(1, curve_order - 1)
randsp = lambda: randint(1, field_modulus - 1)
sbmul = lambda s: multiply(G1, asint(s))
hashsn = lambda *x: hashs(*x) % curve_order
hashpn = lambda *x: hashsn(*[item.n for sublist in x for item in sublist])
hashp = lambda *x: hashs(*[item.n for sublist in x for item in sublist])
addmodn = lambda x, y: (x + y) % curve_order
addmodp = lambda x, y: (x + y) % field_modulus
mulmodn = lambda x, y: (x * y) % curve_order
mulmodp = lambda x, y: (x * y) % field_modulus
submodn = lambda x, y: (x - y) % curve_order
submodp = lambda x, y: (x - y) % field_modulus
# invmodn = lambda x: inv(x, curve_order)
# invmodp = lambda x: inv(x, field_modulus)
negp = lambda x: (x[0], -x[1])


def evalcurve(x):
	a = 5472060717959818805561601436314318772174077789324455915672259473661306552146
	beta = addmodp(mulmodp(mulmodp(x, x), x), 3)
	y = powmod(beta, a, field_modulus)
	return (beta, y)


def isoncurve(x, y):
	beta = addmodp(mulmodp(mulmodp(x, x), x), 3)
	return beta == mulmodp(y, y)


def hashtopoint(x: int):
	x = x % curve_order
	while True:
		beta, y = evalcurve(x)
		if beta == mulmodp(y, y):
			assert isoncurve(x, y)
			return FQ(x), FQ(y)
		x = addmodn(x, 1)



In [12]:
from __future__ import print_function

"""
taghis implements a linkable variant of the AOS 1-out-of-n ring signature
which requires only `n+1` scalars to validate in addition to the `n` public keys.

For more information, see:

 - https://eprint.iacr.org/2004/027.pdf

"""

def uaosring_randkeys(n):
	skeys = [randsn() for _ in range(0, n)]
	pkeys = [sbmul(sk) for sk in skeys]
	return pkeys, skeys

def uaosring_sign(pkeys, mypair, message, tees=None):
	assert len(pkeys) > 0
	mypk, mysk = mypair
	myidx = pkeys.index(mypk)

	tees = tees or [randsn() for _ in range(0, len(pkeys))]
	cees = [0 for _ in range(0, len(pkeys))]
	alpha = randsn()

	M = hashtopoint(message)
	L = hashtopoint(pkeys_hash_calculator(pkeys))
	tag = multiply(L, mysk)
	h = hashp(M, tag)

	for n, i in [(n, (myidx+n) % len(pkeys)) for n in range(0, len(pkeys))]:
		Y = pkeys[i]
		t = tees[i]
		c = alpha if n == 0 else cees[i-1]

		a = add(sbmul(t), multiply(Y, c))
		b = add(multiply(L, t), multiply(tag, c))
		cees[i] = hashs(h, hashp(tag, a, b))

	alpha_gap = submodn(alpha, cees[myidx-1])
	tees[myidx] = addmodn(tees[myidx], mulmodn(mysk, alpha_gap))

	return pkeys, tag, tees, cees[-1]

def uaosring_check(pkeys, tag, tees, seed, message):
	assert len(pkeys) > 0
	assert len(tees) == len(pkeys)
	L = hashtopoint(pkeys_hash_calculator(pkeys))
	M = hashtopoint(message)
	h = hashp(M, tag)
	c = seed
	for i, y in enumerate(pkeys):
		t = tees[i]
		a = add(sbmul(t), multiply(y, c))
		b = add(multiply(L, t), multiply(tag, c))
		c = hashs(h, hashp(tag, a, b))
	return c == seed

def pkeys_hash_calculator(pkeys):
	assert len(pkeys) > 0
	hash_acc = hashs(pkeys[0][0].n)
	for pk in pkeys[1:len(pkeys)]:
		hash_acc = hashs(hash_acc, pk[0].n)
	return hash_acc


if __name__ == "__main__":
	randsn = lambda: randint(1, curve_order - 1)

	msg = randsn()
	pkeys, skeys = uaosring_randkeys(4)
		
	_, tags, tees, cees = uaosring_sign(pkeys=pkeys, mypair =(pkeys[0], skeys[0]), message=msg)

	print(uaosring_check(pkeys=pkeys, tag = tags, tees = tees, seed = cees, message=msg))



61963361044182861068983572895198007383542471015563613741139515008271231006912 61963361044182861068983572895198007383542471015563613741139515008271231006912
True
tags (5326011553673948314162162740218273151433109352020738814241120136823275172731, 8723182156048387243158380979089233321205425354929803677422606735414090903660)
tees [3061865520867786819657328359661165168336444599605390500432034665065838891216, 1704698438504916855447118637202090644562219436573507060237877963725833311164, 17106850840243989338823349043451620794371209087713253851005663660215343844323, 14499418107723575611617266053133834738145634168847450334107371611669585856102]
cees 61963361044182861068983572895198007383542471015563613741139515008271231006912
42883751224339937654788392616662545732046152328799137686306916425819340977921 42883751224339937654788392616662545732046152328799137686306916425819340977921
True
tags (6117910969084899701380456685546986088210568244311347438411601177613796455293, 28555123231451649992029074685

In [30]:
pkeys, skeys = uaosring_randkeys(4)


def uaosring_sign(pkeys, mypair, message, tees=None):
	assert len(pkeys) > 0
	mypk, mysk = mypair
	myidx = pkeys.index(mypk)

	tees = tees or [randsn() for _ in range(0, len(pkeys))]
	cees = [0 for _ in range(0, len(pkeys))]
	alpha = randsn()

	M = hashtopoint(message)
	L = hashtopoint(pkeys_hash_calculator(pkeys))
	
	tag = multiply(L, mysk)
	fake_tag = multiply(L, skeys[0])
	
	h = hashp(M, fake_tag)

	for n, i in [(n, (myidx+n) % len(pkeys)) for n in range(0, len(pkeys))]:
		Y = pkeys[i]
		t = tees[i]
		c = alpha if n == 0 else cees[i-1]

		a = add(sbmul(t), multiply(Y, c))
		b = add(multiply(L, t), multiply(fake_tag, c))
		cees[i] = hashs(h, hashp(fake_tag, a, b))

	alpha_gap = submodn(alpha, cees[myidx-1])
	tees[myidx] = addmodn(tees[myidx], mulmodn(mysk, alpha_gap))

	return pkeys, fake_tag, tees, cees[-1]


_, tags1, tees, cees = uaosring_sign(pkeys=pkeys, mypair =(pkeys[0], skeys[0]), message=123)
print(uaosring_check(pkeys=pkeys, tag = tags1, tees = tees, seed = cees, message=123))
print('tags', tags1)
print('tees', tees)
print('cees', cees)

_, tags2, tees, cees = uaosring_sign(pkeys=pkeys, mypair =(pkeys[1], skeys[1]), message=123)
print(uaosring_check(pkeys=pkeys, tag = tags2, tees = tees, seed = cees, message=123))
print('tags', tags2)
print('tees', tees)
print('cees', cees)

print(uaosring_check(pkeys=pkeys, tag = tags1, tees = tees, seed = cees, message=123))
print('tags', tags1)
print('tees', tees)
print('cees', cees)

29128122182249769720471860434446980808500673885812063284846004336265015971798 29128122182249769720471860434446980808500673885812063284846004336265015971798
True
tags (5465241287303899619765327258666760237756334491980357140777593919062617657502, 2838853629268772779002823315551341503546866172082134782292514553778785378835)
tees [20180328459793893183783085037932531025294362539731397702885190003951397259111, 17472002177205726329971357742600739163687147845255900071539273939335396566098, 6163175521751607112897585151993118401167239778093672024165900140387230116079, 15062208288195856949809299902242313448469175702917327328709986661930580463974]
cees 29128122182249769720471860434446980808500673885812063284846004336265015971798
111371078269578645520277156928483900152679215664191203942958850484535989039836 59965191443011130664638963675386954620085709086404338464820730933221060669003
False
tags (5465241287303899619765327258666760237756334491980357140777593919062617657502, 28388536292687727790028233